In [1]:
# Dir of my huggingface models
# https://huggingface.co/models?sort=modified&search=richardchai%2F

In [2]:
import pandas as pd
import torch
from transformers import pipeline

from pathlib import Path
import sys
cwd = Path.cwd()
common_code_path = cwd.parent.parent / "plp_common_code"

if str(common_code_path) not in sys.path:
    sys.path.append(str(common_code_path))

from clf_Helper import load_hf_pdt_clf

# 1.0 Load Classifier Models
- pdt_clf
- action_clf
- sentiment_clf

In [3]:
input_text = ['please tell me more about your fixed deposit.', 
              'your savings rate is terrible!', 
              'OMG, I made a ton in the currency carry trade!']

In [4]:
pdt_clf = load_hf_pdt_clf(
    nlp_task='text-classification', 
    hf_model_chkpt="richardchai/plp_pdt_clr_tinybert")

response = pdt_clf(input_text)
for r in response:
    print(r['label'])

fixed_deposit
savings_account
forex_account


In [5]:
action_clf = load_hf_pdt_clf(
    nlp_task='text-classification', 
    hf_model_chkpt="richardchai/plp_action_clr_distilbert")
action_clf(input_text)

[{'label': 'find', 'score': 0.16672246158123016},
 {'label': 'redeem', 'score': 0.06389164924621582},
 {'label': 'refund', 'score': 0.11818492412567139}]

In [6]:
sentiment_clf = load_hf_pdt_clf(
    nlp_task='text-classification', 
    hf_model_chkpt="richardchai/plp_sentiment_clr_distilbert")
sentiment_clf(input_text)

[{'label': 'positive', 'score': 0.9965279698371887},
 {'label': 'negative', 'score': 0.9875072836875916},
 {'label': 'positive', 'score': 0.8937973380088806}]

# Load `ccd_unlabeled_final.parquet` (this dataset is already data prepared)
- Notes: This is not the common function which needs to convert multi-turn to singles statements of customer uttrance.
- this file ccd_unlabled_final has already been data prep. So I now use it to create Aspect Topic and save to Topics Database

In [7]:
df = pd.read_parquet("data_input/ccd_unlabeled_final.parquet")
df.shape, df.columns

((13974, 3), Index(['date', 'dlg_id', 'text'], dtype='object'))

# Create Aspect Topic

In [8]:
# def predict_product(x: [str, list]):
#     response = pdt_clf(x)
#     pdt = [r['label'] for r in response]
#     # return pdt
#     if isinstance(pdt, list) and len(pdt) < 2:
#         return pdt[0]
#     else:
#         return str(pdt)[1:-1]

# # predict_product(input_text)
# predict_product("I have 3 savings account.")

In [9]:
def predict_product(x: [str, list]):
    response = pdt_clf(x)
    product = [r['label'] for r in response]
    if isinstance(product, list) and len(product) < 2:
        return product[0]
    else:
        return str(product)[1:-1]
    
df['product'] = df['text'].apply(lambda x: predict_product(x))
display(df.head())
display(df.tail())

date              dlg_id  \
0  2024-10-05  2024-10-07_1017--0   
1  2024-10-05  2024-10-07_1017--1   
2  2024-10-05  2024-10-07_1017--2   
3  2024-10-05  2024-10-07_1017--3   
4  2024-10-05  2024-10-07_1017--4   

                                                text product  
0                      i am still waiting on my card    card  
1  what can i do if my card still hasnt arrived a...    card  
2  i have been waiting over a week is the card st...    card  
3  can i track my card while it is in the process...    card  
4  how do i know if i will get my card or if it i...    card

date               dlg_id  \
13969  2024-10-05  2024-10-14__dlg_991   
13970  2024-10-05  2024-10-14__dlg_991   
13971  2024-10-05  2024-10-14__dlg_992   
13972  2024-10-05  2024-10-14__dlg_992   
13973  2024-10-05  2024-10-14__dlg_992   

                                                    text     product  
13969  Yes, I have. I’m logged in right now, but I’m ...     website  
13970  Thanks, Jordan. I found it. I’ll try going thr...  debit_card  
13971  Hi Alex, I hope you're doing well. I'm very im...        loan  
13972  Sure, the website is [Website Name]. While the...        loan  
13973  That would be perfect. Please go ahead and set...  debit_card

In [10]:
def predict_action(x: [str, list]):
    response = action_clf(x)
    action = [r['label'] for r in response]
    
    if isinstance(action, list) and len(action) < 2:
        return action[0]
    else:
        return str(action)[1:-1]
    
df['action'] = df['text'].apply(lambda x: predict_action(x))
display(df.head())
display(df.tail())

date              dlg_id  \
0  2024-10-05  2024-10-07_1017--0   
1  2024-10-05  2024-10-07_1017--1   
2  2024-10-05  2024-10-07_1017--2   
3  2024-10-05  2024-10-07_1017--3   
4  2024-10-05  2024-10-07_1017--4   

                                                text product   action  
0                      i am still waiting on my card    card  inquire  
1  what can i do if my card still hasnt arrived a...    card  inquire  
2  i have been waiting over a week is the card st...    card  inquire  
3  can i track my card while it is in the process...    card  inquire  
4  how do i know if i will get my card or if it i...    card  inquire

date               dlg_id  \
13969  2024-10-05  2024-10-14__dlg_991   
13970  2024-10-05  2024-10-14__dlg_991   
13971  2024-10-05  2024-10-14__dlg_992   
13972  2024-10-05  2024-10-14__dlg_992   
13973  2024-10-05  2024-10-14__dlg_992   

                                                    text     product    action  
13969  Yes, I have. I’m logged in right now, but I’m ...     website  exchange  
13970  Thanks, Jordan. I found it. I’ll try going thr...  debit_card   unblock  
13971  Hi Alex, I hope you're doing well. I'm very im...        loan     block  
13972  Sure, the website is [Website Name]. While the...        loan   dispute  
13973  That would be perfect. Please go ahead and set...  debit_card    access

In [11]:
def predict_sentiment(x: [str, list]):
    response = sentiment_clf(x)
    sentiment = [r['label'] for r in response]
    
    if isinstance(sentiment, list) and len(sentiment) < 2:
        return sentiment[0]
    else:
        return str(sentiment)[1:-1]
    
df['sentiment'] = df['text'].apply(lambda x: predict_sentiment(x))
display(df.head())
display(df.tail())

date              dlg_id  \
0  2024-10-05  2024-10-07_1017--0   
1  2024-10-05  2024-10-07_1017--1   
2  2024-10-05  2024-10-07_1017--2   
3  2024-10-05  2024-10-07_1017--3   
4  2024-10-05  2024-10-07_1017--4   

                                                text product   action  \
0                      i am still waiting on my card    card  inquire   
1  what can i do if my card still hasnt arrived a...    card  inquire   
2  i have been waiting over a week is the card st...    card  inquire   
3  can i track my card while it is in the process...    card  inquire   
4  how do i know if i will get my card or if it i...    card  inquire   

  sentiment  
0   neutral  
1   neutral  
2   neutral  
3  positive  
4  positive

date               dlg_id  \
13969  2024-10-05  2024-10-14__dlg_991   
13970  2024-10-05  2024-10-14__dlg_991   
13971  2024-10-05  2024-10-14__dlg_992   
13972  2024-10-05  2024-10-14__dlg_992   
13973  2024-10-05  2024-10-14__dlg_992   

                                                    text     product  \
13969  Yes, I have. I’m logged in right now, but I’m ...     website   
13970  Thanks, Jordan. I found it. I’ll try going thr...  debit_card   
13971  Hi Alex, I hope you're doing well. I'm very im...        loan   
13972  Sure, the website is [Website Name]. While the...        loan   
13973  That would be perfect. Please go ahead and set...  debit_card   

         action      sentiment  
13969  exchange       positive  
13970   unblock       positive  
13971     block  very positive  
13972   dispute  very negative  
13973    access  very negative

# Calculate the ABSA (Aspect Topic)
- Aspect Topic is defined as “customer action (intent) + product/solution”

In [12]:
df['aspect_topic'] = df['action'] + '__' + df['product']
display(df.head())
display(df.tail())

date              dlg_id  \
0  2024-10-05  2024-10-07_1017--0   
1  2024-10-05  2024-10-07_1017--1   
2  2024-10-05  2024-10-07_1017--2   
3  2024-10-05  2024-10-07_1017--3   
4  2024-10-05  2024-10-07_1017--4   

                                                text product   action  \
0                      i am still waiting on my card    card  inquire   
1  what can i do if my card still hasnt arrived a...    card  inquire   
2  i have been waiting over a week is the card st...    card  inquire   
3  can i track my card while it is in the process...    card  inquire   
4  how do i know if i will get my card or if it i...    card  inquire   

  sentiment   aspect_topic  
0   neutral  inquire__card  
1   neutral  inquire__card  
2   neutral  inquire__card  
3  positive  inquire__card  
4  positive  inquire__card

date               dlg_id  \
13969  2024-10-05  2024-10-14__dlg_991   
13970  2024-10-05  2024-10-14__dlg_991   
13971  2024-10-05  2024-10-14__dlg_992   
13972  2024-10-05  2024-10-14__dlg_992   
13973  2024-10-05  2024-10-14__dlg_992   

                                                    text     product  \
13969  Yes, I have. I’m logged in right now, but I’m ...     website   
13970  Thanks, Jordan. I found it. I’ll try going thr...  debit_card   
13971  Hi Alex, I hope you're doing well. I'm very im...        loan   
13972  Sure, the website is [Website Name]. While the...        loan   
13973  That would be perfect. Please go ahead and set...  debit_card   

         action      sentiment         aspect_topic  
13969  exchange       positive    exchange__website  
13970   unblock       positive  unblock__debit_card  
13971     block  very positive          block__loan  
13972   dispute  very negative        dispute__loan  
13973    access  very negative   access__debit_card

In [15]:
df[['product', 'action', 'aspect_topic']].sample(5)

product  action        aspect_topic
9101   debit_card   renew   renew__debit_card
10712     account    link       link__account
6026   mobile_app  access  access__mobile_app
8558      account   renew      renew__account
13872  debit_card     pay     pay__debit_card

In [13]:
df.columns

Index(['date', 'dlg_id', 'text', 'product', 'action', 'sentiment',
       'aspect_topic'],
      dtype='object')

# Save DF as the topics database
- If dashboard does ABSA, they can use the aspect topic
- if dashboard does topic modeling, they can just drop whatever columns they don't need
    - basically keep only the column 'text' and ask for whatever number of rows they want (based on date or some other criteria), do the topic modelling process on that text
- If this works, then there is only 1 parquet file (simulating a real dbms)
    - then all incoming dialog goes through data preparation and is save into this master topics database
    - but NB:
        - the results of topic modelling (which takes a long time) is save in a separate file/table.
        - the results of ABSA calcuation is calculated on demand and not saved permanently.

In [14]:
df.to_parquet("data_output/topics_db.parquet", index=False)
df.to_csv("data_output/topics_db.csv", index=False)
print("df saved to disk")

df saved to disk
